# Large-Scale Stochastic Variational GP Regression (CUDA)

## Overview

In this notebook, we'll give an overview of how to use SVGP stochastic variational regression ((https://arxiv.org/pdf/1411.2005.pdf)) to rapidly train using minibatches on the `3droad` UCI dataset with hundreds of thousands of training examples. 

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Loading Data

For this example notebook, we'll be using the `song` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a **~136 MB** file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', '3droad.mat')
    
data = torch.Tensor(loadmat('3droad.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Creating a DataLoader

The next step is to create a torch `DataLoader` that will handle getting us random minibatches of data. This involves using the standard `TensorDataset` and `DataLoader` modules provided by PyTorch.

In this notebook we'll be using a fairly large batch size of 1024 just to make optimization run faster, but you could of course change this as you so choose.

In [3]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

## Defining the SVGP Model

We now define the GP regression module that, intuitvely, will act as the final "layer" of our neural network. In this case, because we are doing variational inference and *not* exact inference, we will be using an `AbstractVariationalGP`. In this example, because we will be learning the inducing point locations, we'll be using a base `VariationalStrategy` with `learn_inducing_locations=True`.

Because the feature extractor we defined above extracts two features, we'll need to define our grid bounds over two dimensions.

In [4]:
from gpytorch.models import AbstractVariationalGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import HalfWhitenedVariationalStrategy

class GPModel(AbstractVariationalGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = HalfWhitenedVariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

inducing_points = train_x[:500, :]
model = GPModel(inducing_points=inducing_points).cuda()
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()

## Training the Model

The cell below trains the model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE.

Unlike when using the exact GP marginal log likelihood, performing variational inference allows us to make use of stochastic optimization techniques. For this example, we'll do one epoch of training. Given the small size of the neural network relative to the size of the dataset, this should be sufficient to achieve comparable accuracy to what was observed in the DKL paper.

The optimization loop differs from the one seen in our more simple tutorials in that it involves looping over both a number of training iterations (epochs) *and* minibatches of the data. However, the basic process is the same: for each minibatch, we forward through the model, compute the loss (the `VariationalMarginalLogLikelihood` or ELBO), call backwards, and do a step of optimization.

In [ ]:
import time

model.train()
likelihood.train()

# We'll do 6 epochs of training in this tutorial
num_epochs = 4

# We use SGD here, rather than Adam. Emperically, we find that SGD is better for variational regression
optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.001)

scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[3, 5], gamma=0.1)

# Our loss object. We're using the VariationalELBO, which essentially just computes the ELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0), combine_terms=False)

# We use more CG iterations here because the preconditioner introduced in the NeurIPS paper seems to be less
# effective for VI.
with gpytorch.settings.max_cg_iterations(45):
    for i in range(num_epochs):
        scheduler.step()
        # Within each iteration, we will go over each minibatch of data
        for minibatch_i, (x_batch, y_batch) in enumerate(train_loader):
            start_time = time.time()
            optimizer.zero_grad()
            output = model(x_batch)
            # with combine_terms=False, we get the terms of the ELBO separated so we can print them individually if we'd like.
            # loss = -mll(output, y_batch) would also work.
            log_lik, kl_div, log_prior = mll(output, y_batch)
            loss = -(log_lik - kl_div + log_prior)
            print('Epoch %d [%d/%d] - Loss: %.3f [%.3f, %.3f, %.3f]' % (i + 1, minibatch_i, len(train_loader), loss.item(), log_lik.item(), kl_div.item(), log_prior.item()))

            loss.backward()
            optimizer.step()

/home/jake.gardner/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:82: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


tensor(0., device='cuda:0', grad_fn=<NormBackward0>)
tensor(6.1839e-05, device='cuda:0', grad_fn=<NormBackward0>)
> /home/jake.gardner/git/gpytorch/gpytorch/variational/half_whitened_variational_strategy.py(181)forward()
    179             from IPython.core.debugger import set_trace
    180             set_trace()
--> 181             predictive_covar = data_data_covar + full_part.mul(-1)
    182 
    183             if self.training:

ipdb> covar_cache
tensor([[ 2.0988e-05,  7.2612e-06, -2.1143e-05,  ...,  2.5563e-08,
          1.7376e-05, -5.5743e-06],
        [-2.1663e-05,  2.9096e-05,  5.3868e-05,  ...,  3.9912e-08,
          3.3303e-05,  8.1069e-06],
        [ 1.7505e-04, -1.1799e-04, -3.2948e-04,  ..., -8.7491e-08,
         -4.3049e-05,  6.3796e-05],
        ...,
        [-2.9459e-03,  2.7300e-03, -4.3676e-03,  ...,  1.1121e-01,
         -3.3216e-02, -3.0152e-03],
        [-2.7687e-02,  1.0093e-01,  6.0739e-02,  ..., -4.6503e-02,
          6.4545e-01, -1.2580e-01],
        [-8.35

ipdb> qq
tensor([[-2.3888e-02,  2.0590e-02, -4.0280e-02,  ...,  5.9359e-03,
          6.7584e-02,  4.3529e-02],
        [ 1.5953e-02,  4.1121e-02, -5.7521e-03,  ..., -6.1103e-02,
         -4.7232e-02,  4.4257e-02],
        [ 2.5312e-02,  1.7891e-04, -9.0191e-03,  ..., -6.0704e-02,
         -2.5618e-02,  5.1091e-02],
        ...,
        [-1.8273e-02, -1.7307e-01, -2.1893e-01,  ...,  5.0359e-02,
          2.8332e-02,  2.0349e-02],
        [-6.5715e-01,  2.7181e-02, -1.2818e-02,  ..., -2.2878e-02,
         -4.3554e-02,  6.0219e-02],
        [ 7.9669e-02, -1.0110e-02, -1.8297e-01,  ..., -3.3258e-02,
         -5.4535e-02,  2.4553e-02]], device='cuda:0', grad_fn=<SymeigBackward>)
ipdb> print(qq.shape)
torch.Size([500, 500])
ipdb> DiagLazyTensor
<class 'gpytorch.lazy.diag_lazy_tensor.DiagLazyTensor'>
ipdb> ss = DiagLazyTensor(ss).evaluate()
ipdb> ss
tensor([[-6.1204e-06, -0.0000e+00, -0.0000e+00,  ..., -0.0000e+00,
         -0.0000e+00, -0.0000e+00],
        [-0.0000e+00, -4.2017e-06, -0.000

ipdb> qqq = torch.triangular_solve(inn, L, transpose=True)
ipdb> qqq
torch.return_types.triangular_solve(
solution=tensor([[ 0.0000e+00, -1.0279e-09,  1.1619e-08,  ...,  0.0000e+00,
          5.7233e-07, -2.1462e-07],
        [-1.0344e-09,  0.0000e+00,  1.6145e-07,  ...,  0.0000e+00,
          1.1519e-06,  2.8797e-07],
        [ 3.1366e-08,  4.3309e-07, -7.7250e-07,  ...,  0.0000e+00,
          1.5450e-06,  3.0900e-06],
        ...,
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  5.4279e-03,
         -1.1885e-03, -5.7980e-05],
        [ 1.4548e-05,  2.9097e-05,  1.4548e-05,  ..., -1.4243e-03,
          2.2172e-02, -3.5437e-03],
        [-4.6442e-06,  6.1923e-06,  2.4769e-05,  ..., -5.9150e-05,
         -3.0167e-03,  1.7369e-03]], device='cuda:0',
       grad_fn=<TriangularSolveBackward>),
cloned_coefficient=tensor([[ 8.3315e-01,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 9.3213e-02,  8.2792e-01,  0.0000e+00,  ...,  0.0000e+00,
    

In [14]:
%time output = model(x_batch)

Compiling libKeOpstorch8dcd847cfd in /home/jake.gardner/.cache/pykeops-1.1.2-cpython-37//build-libKeOpstorch8dcd847cfd:
       formula: Sum_Reduction((Exp((Minus(Sum(Square((Var(0,3,0) - Var(1,3,1))))) / IntCst(2))) * Var(2,1,1)),0)
       aliases: Var(0,3,0); Var(1,3,1); Var(2,1,1); 
       dtype  : float32
... 
--------------------- MAKE DEBUG -----------------
Command '['cmake', '--build', '.', '--target', 'libKeOpstorch8dcd847cfd', '--', 'VERBOSE=1']' returned non-zero exit status 2.
/usr/local/bin/cmake -S/home/jake.gardner/anaconda3/lib/python3.7/site-packages/pykeops -B/home/jake.gardner/.cache/pykeops-1.1.2-cpython-37/build-libKeOpstorch8dcd847cfd --check-build-system CMakeFiles/Makefile.cmake 0
/usr/bin/make -f CMakeFiles/Makefile2 libKeOpstorch8dcd847cfd
make[1]: Entering directory '/home/jake.gardner/.cache/pykeops-1.1.2-cpython-37/build-libKeOpstorch8dcd847cfd'
/usr/local/bin/cmake -S/home/jake.gardner/anaconda3/lib/python3.7/site-packages/pykeops -B/home/jake.gardner/.cach

ModuleNotFoundError: No module named 'libKeOpstorch8dcd847cfd'

## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`). Because the test set is substantially smaller than the training set, we don't need to make predictions in mini batches here, although this can be done by passing in minibatches of `test_x` rather than the full tensor.

In [11]:
model.eval()
likelihood.eval()
means = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        means = torch.cat([means, preds.mean.cpu()])
means = means[1:]

/home/jake.gardner/git/gpytorch/gpytorch/lazy/cached_cg_lazy_tensor.py:161: UserWarning: CachedCGLazyTensor had to run CG on a tensor of size torch.Size([500, 1918]). For best performance, this LazyTensor should pre-register all vectors to run CG against.
  "LazyTensor should pre-register all vectors to run CG against.".format(rhs.shape)


In [13]:
print('Test MAE: {}'.format(torch.mean(torch.abs(means - test_y.cpu()))))

Test MAE: 9.761589050292969
